# Modeling

## Hans

## Annie: KNN and gradient boosting classifier

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import metrics
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from skopt import BayesSearchCV

import datetime

ModuleNotFoundError: No module named 'skopt'

In [2]:
df = pd.read_csv('../Data/salary_cleaned.csv')
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,year,month,year_month,timestamp_3mos,year_month_3mos,state_short,inflation_rate,inflation_rate_3mos,state,employment_rate,employment_rate_3mos
0,2018-06-03 13:58:20,Yahoo,IC2,Software Engineer,160.5,"Sunnyvale, CA",0.58,0.58,Full Stack,2018,6,2018-06,2018-03-03 13:58:20,2018-03,CA,0.029,0.024,California,0.957660,0.956797
1,2018-06-04 20:28:22,Facebook,E3,Software Engineer,165.0,"Seattle, WA",1.00,1.00,Full Stack,2018,6,2018-06,2018-03-04 20:28:22,2018-03,WA,0.029,0.024,Washington,0.955998,0.954978
2,2018-06-05 00:56:33,VmWare,Senior MTS,Software Engineer,218.0,"Palo Alto, CA",8.00,1.00,Distributed Systems (Back-End),2018,6,2018-06,2018-03-05 00:56:33,2018-03,CA,0.029,0.024,California,0.957660,0.956797
3,2018-06-05 01:19:05,Uber,L4,Software Engineer,240.0,"San Francisco, CA",3.00,0.00,Web Development (Front-End),2018,6,2018-06,2018-03-05 01:19:05,2018-03,CA,0.029,0.024,California,0.957660,0.956797
4,2018-06-05 07:13:17,Capital One,Master Software Engineer,Software Engineer,196.0,"New York, NY",8.00,2.00,iOS,2018,6,2018-06,2018-03-05 07:13:17,2018-03,NY,0.029,0.024,New York,0.959053,0.955962


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24496 entries, 0 to 24495
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                24496 non-null  object 
 1   company                  24496 non-null  object 
 2   level                    24496 non-null  object 
 3   title                    24496 non-null  object 
 4   totalyearlycompensation  24496 non-null  float64
 5   location                 24496 non-null  object 
 6   yearsofexperience        24496 non-null  float64
 7   yearsatcompany           24496 non-null  float64
 8   tag                      24496 non-null  object 
 9   year                     24496 non-null  int64  
 10  month                    24496 non-null  int64  
 11  year_month               24496 non-null  object 
 12  timestamp_3mos           24496 non-null  object 
 13  year_month_3mos          24496 non-null  object 
 14  state_short           

In [4]:
# convert year and month from integer to string
df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24496 entries, 0 to 24495
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                24496 non-null  object 
 1   company                  24496 non-null  object 
 2   level                    24496 non-null  object 
 3   title                    24496 non-null  object 
 4   totalyearlycompensation  24496 non-null  float64
 5   location                 24496 non-null  object 
 6   yearsofexperience        24496 non-null  float64
 7   yearsatcompany           24496 non-null  float64
 8   tag                      24496 non-null  object 
 9   year                     24496 non-null  object 
 10  month                    24496 non-null  object 
 11  year_month               24496 non-null  object 
 12  timestamp_3mos           24496 non-null  object 
 13  year_month_3mos          24496 non-null  object 
 14  state_short           

In [5]:
df['location'].value_counts(normalize=True)

Seattle, WA          0.161169
San Francisco, CA    0.136920
New York, NY         0.078829
Redmond, WA          0.065072
Sunnyvale, CA        0.049110
                       ...   
Schenectady, NY      0.000041
Allentown, PA        0.000041
Portsmouth, RI       0.000041
Fullerton, CA        0.000041
Elk Grove, CA        0.000041
Name: location, Length: 447, dtype: float64

In [6]:
# create 
features_all = ['company', 'title', 'location', 'yearsofexperience', 'yearsatcompany', 'year', 'month', 'year_month', 'state_short', 'inflation_rate', 'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos']
features_short = ['company', 'title', 'yearsofexperience', 'yearsatcompany', 'year', 'month', 'state', 'inflation_rate', 'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos']

X = df[features_short]
y = df['totalyearlycompensation']
X.shape, y.shape

((24496, 11), (24496,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((18372, 11), (18372,), (6124, 11), (6124,))

In [8]:
X_train.head()

,company,title,yearsofexperience,yearsatcompany,year,month,state,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
19069,Google,Product Manager,15.0,1.0,2020,7,Washington,0.010,0.003,0.898081,0.837230
3156,Amazon,Software Engineer,5.0,5.0,2019,3,Washington,0.019,0.019,0.955238,0.954989
14183,Twilio,Software Engineer,4.0,1.0,2020,4,California,0.003,0.025,0.836228,0.961390
13104,Intel,Software Engineer,9.0,9.0,2020,3,Arizona,0.015,0.023,0.938761,0.954570
20679,Cisco,Software Engineer,0.0,0.0,2020,8,California,0.013,0.001,0.888158,0.835571


### Transform data: standardize and one hot encoding

In [26]:
ct = ColumnTransformer([
    ('sc', StandardScaler(), make_column_selector(dtype_include=np.number)),
    #('ohe', OneHotEncoder(handle_unknown='ignore'), make_column_selector(dtype_include=object)),
    #('ohe', OneHotEncoder(handle_unknown='ignore'), ['company','title','year','month','state']),
    ('ohe', OneHotEncoder(handle_unknown='ignore'), [0,1,6])
    ])

X_train_ct = ct.fit_transform(X_train)
X_test_ct = ct.transform(X_test)

In [42]:
X_train_ct.shape

(18372, 1219)

In [25]:
X_train

,company,title,yearsofexperience,yearsatcompany,year,month,state,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
19069,Google,Product Manager,15.0,1.0,2020,7,Washington,0.010,0.003,0.898081,0.837230
3156,Amazon,Software Engineer,5.0,5.0,2019,3,Washington,0.019,0.019,0.955238,0.954989
14183,Twilio,Software Engineer,4.0,1.0,2020,4,California,0.003,0.025,0.836228,0.961390
13104,Intel,Software Engineer,9.0,9.0,2020,3,Arizona,0.015,0.023,0.938761,0.954570
20679,Cisco,Software Engineer,0.0,0.0,2020,8,California,0.013,0.001,0.888158,0.835571
...,...,...,...,...,...,...,...,...,...,...,...
21575,BlackRock,Product Manager,8.0,2.0,2020,8,California,0.013,0.001,0.888158,0.835571
5390,Amazon,Software Engineer,2.0,0.0,2019,7,Washington,0.018,0.020,0.957501,0.955663
860,Splunk,Product Manager,9.0,1.0,2018,9,Washington,0.023,0.029,0.956046,0.955998
15795,Facebook,Software Engineer,5.0,1.0,2020,5,California,0.001,0.023,0.835571,0.961092


In [22]:
ct.get_feature_names_out()

array(['sc__yearsofexperience', 'sc__yearsatcompany',
       'sc__inflation_rate', ..., 'ohe__state_Washington',
       'ohe__state_West Virginia', 'ohe__state_Wisconsin'], dtype=object)

### KNN 

In [38]:
knn = KNeighborsRegressor(n_jobs=-1)

knn_params = {
    'n_neighbors': [3,5,7,9,11,13,15,17,19]
    ,'weights': ['uniform', 'distance']
    ,'p': [1,2]   # this one makes the training time much longer, and didn't improve R2 significantly
}

gs_knn = GridSearchCV(estimator = knn,
                     param_grid = knn_params,
                     cv = 5)

# train the model

print(datetime.datetime.now())

gs_knn.fit(X_train_ct, y_train)

print(datetime.datetime.now())
print()

2022-01-04 12:09:34.704417
2022-01-04 12:31:22.106470



In [ ]:
# predict
knn_pred_train = gs_knn.predict(X_train_ct)
knn_pred_test = gs_knn.predict(X_test_ct)

# evaluate
gs_knn_R2_train = gs_knn.score(X_train_ct, y_train)
gs_knn_R2_test = gs_knn.score(X_test_ct, y_test)

print(f'gs_knn_R2_train: {round(gs_knn_R2_train, 4)}')
print(f'gs_knn_R2_test: {round(gs_knn_R2_test, 4)}')

In [ ]:
gs_knn_mse_train = metrics.mean_squared_error(y_train, knn_pred_train)
gs_knn_mse_test = metrics.mean_squared_error(y_test, knn_pred_test)

print(f'gs_knn_mse_train: {round(gs_knn_mse_train, 4)}')
print(f'gs_knn_mse_test: {round(gs_knn_mse_test, 4)}')

In [ ]:
perf_dict = {
    'R2':[gs_knn_R2_train, gs_knn_R2_test],
    'mse':[gs_knn_mse_train, gs_knn_mse_test]
}

In [ ]:
perf_df = pd.DataFrame(perf_dict, index=['Train','Test'])


In [40]:
gs_knn.best_estimator_

KNeighborsRegressor(n_jobs=-1, n_neighbors=13, p=1, weights='distance')

In [ ]:
gs_knn.cv_results_

In [ ]:
pd.DataFrame(gs_knn.cv_results_)

**Observation**: 
1. Comparing the R2 score and mse between the training and testing sets, the model is very overfit.
2. The best model from grid search CV uses 13 nearest neighbors, p=1, and distance as weights.

### Graident Boosting Classifer

#### No grid search

In [54]:
# fit the model
gbr = GradientBoostingRegressor(n_estimators=400,
                                max_depth=3,
                                max_features=800,
                                random_state=42)

print(datetime.datetime.now())

gbr.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-05 15:52:09.839168
2022-01-05 15:54:41.358522


In [58]:
# predict
gbr_pred_train = gbr.predict(X_train_ct)
gbr_pred_test = gbr.predict(X_test_ct)

# evaluate
gbr_R2_train = gbr.score(X_train_ct, y_train)
gbr_R2_test = gbr.score(X_test_ct, y_test)

gbr_mse_train = metrics.mean_squared_error(y_train, gbr_pred_train)
gbr_mse_test = metrics.mean_squared_error(y_test, gbr_pred_test)

gbr_perf_dict = {
    'R2': [gbr_R2_train, gbr_R2_test],
    'MSE': [gbr_mse_train, gbr_mse_test]
}

gbr_perf_df = pd.DataFrame(gbr_perf_dict, index=['Train','Test'])
gbr_perf_df.T

,Train,Test
R2,0.597249,0.531825
MSE,6834.117462,8198.404325


#### GridSearchCV

##### GridSearch 0 
Model No. 0: killed after running for 19 hours

##### GridSearch 1

In [69]:
# build the grid search for hyperparameters 

gbr = GradientBoostingRegressor(random_state=42)

gbr_params = {
    'n_estimators': [300,400,500],
    #'learning_rate': [0.01, 0.1],
    'max_depth': [5,6,7,8],
    #'min_samples_split': [5,7,10],
    #'min_samples_leaf': [2,3,5],
    'max_features': [100,150,200]
}

gs_gbr = GridSearchCV(gbr,
                     gbr_params,
                     cv=5)

# train

print(datetime.datetime.now())

gs_gbr.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-05 21:12:19.908880
2022-01-05 23:36:23.269022


In [ ]:
# predict

gs_gbr_pred_train = gs_gbr.predict(X_train_ct)
gs_gbr_pred_test = gs_gbr.predict(X_test_ct)

# evaluate

gs_gbr_R2_train = gs_gbr.score(X_train_ct, y_train)
gs_gbr_R2_test = gs_gbr.score(X_test_ct, y_test)

print(f'gs_gbr_R2_train: {round(gs_gbr_R2_train), 4}')
print(f'gs_gbr_R2_test: {round(gs_gbr_R2_test), 4}')
print()

gs_gbr_mse_train = metrics.mean_squared_error(y_train, gs_gbr_pred_train)
gs_gbr_mse_test = metrics.mean_squared_error(y_test, gs_gbr_pred_test)

print(f'gs_gbr_mse_train: {round(gs_gbr_mse_train), 4}')
print(f'gs_gbr_mse_test: {round(gs_gbr_mse_test), 4}')

gs_gbr_perf_dict = {
    'R2': [gs_gbr_R2_train, gs_gbr_R2_test],
    'MSE': [gs_gbr_mse_train, gs_gbr_mse_test]
}

gs_gbr_perf_df = pd.DataFrame(gs_gbr_perf_dict, index=['Train','Test'])
gs_gbr_perf_df.T

gs_gbr_R2_train: (1.0, 4)
gs_gbr_R2_test: (1.0, 4)

gs_gbr_mse_train: (3932.0, 4)
gs_gbr_mse_test: (7887.0, 4)


,Train,Test
R2,0.768261,0.549606
MSE,3932.287030,7887.025790


In [ ]:
gs_gbr.best_estimator_

GradientBoostingRegressor(max_depth=7, max_features=200, n_estimators=500,
                          random_state=42)

In [ ]:
gs_gbr.best_score_

0.5460109618022845

In [ ]:
pd.DataFrame(gs_gbr.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,16.568261,0.238732,0.032683,0.004730,5,100,300,"{'max_depth': 5, 'max_features': 100, 'n_estim...",0.444031,0.551792,0.547701,0.546139,0.580407,0.534014,0.046690,36
1,22.243769,0.167745,0.038723,0.010096,5,100,400,"{'max_depth': 5, 'max_features': 100, 'n_estim...",0.448325,0.555871,0.546635,0.549169,0.584890,0.536978,0.046376,33
2,27.666378,0.212298,0.039335,0.001972,5,100,500,"{'max_depth': 5, 'max_features': 100, 'n_estim...",0.453342,0.556975,0.546767,0.550798,0.588266,0.539229,0.045358,27
3,24.047348,0.107979,0.027109,0.006487,5,150,300,"{'max_depth': 5, 'max_features': 150, 'n_estim...",0.440484,0.553796,0.541597,0.550436,0.584208,0.534104,0.048961,35
4,32.512880,0.435903,0.032332,0.001420,5,150,400,"{'max_depth': 5, 'max_features': 150, 'n_estim...",0.449931,0.556597,0.544277,0.555669,0.585043,0.538303,0.046187,30
5,40.733250,0.219907,0.042235,0.005071,5,150,500,"{'max_depth': 5, 'max_features': 150, 'n_estim...",0.451308,0.558210,0.544387,0.558260,0.588900,0.540213,0.046779,25
6,31.680012,0.079360,0.024570,0.000818,5,200,300,"{'max_depth': 5, 'max_features': 200, 'n_estim...",0.439142,0.561392,0.548560,0.547625,0.588163,0.536977,0.051060,34
7,42.284636,0.519523,0.033507,0.003949,5,200,400,"{'max_depth': 5, 'max_features': 200, 'n_estim...",0.440899,0.565253,0.550560,0.555386,0.591301,0.540680,0.051839,24
8,53.126462,0.314038,0.038191,0.005210,5,200,500,"{'max_depth': 5, 'max_features': 200, 'n_estim...",0.443108,0.568328,0.553694,0.560033,0.593705,0.543774,0.052139,11
9,19.256413,0.143803,0.026260,0.001714,6,100,300,"{'max_depth': 6, 'max_features': 100, 'n_estim...",0.439803,0.563794,0.551821,0.550353,0.585639,0.538282,0.050834,31


##### GridSearch 2

In [66]:
# build the grid search for hyperparameters 

gbr2 = GradientBoostingRegressor(random_state=42)

gbr_params = {
    'n_estimators': [400],
    #'learning_rate': [0.01, 0.1],
    'max_depth': [3,4,5,6],
    #'min_samples_split': [5,7,10],
    #'min_samples_leaf': [2,3,5],
    'max_features': [200, 300, 400]
}

gs_gbr2 = GridSearchCV(gbr,
                     gbr_params,
                     cv=5)

# train

print(datetime.datetime.now())

gs_gbr2.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-05 18:54:09.653189
2022-01-05 19:58:42.168533


In [67]:
# predict

gs_gbr2_pred_train = gs_gbr2.predict(X_train_ct)
gs_gbr2_pred_test = gs_gbr2.predict(X_test_ct)

# evaluate

gs_gbr2_R2_train = gs_gbr2.score(X_train_ct, y_train)
gs_gbr2_R2_test = gs_gbr2.score(X_test_ct, y_test)

print(f'gs_gbr2_R2_train: {round(gs_gbr2_R2_train), 4}')
print(f'gs_gbr2_R2_test: {round(gs_gbr2_R2_test), 4}')
print()

gs_gbr2_mse_train = metrics.mean_squared_error(y_train, gs_gbr2_pred_train)
gs_gbr2_mse_test = metrics.mean_squared_error(y_test, gs_gbr2_pred_test)

print(f'gs_gbr2_mse_train: {round(gs_gbr2_mse_train), 4}')
print(f'gs_gbr2_mse_test: {round(gs_gbr2_mse_test), 4}')

gs_gbr2_perf_dict = {
    'R2': [gs_gbr2_R2_train, gs_gbr2_R2_test],
    'MSE': [gs_gbr2_mse_train, gs_gbr2_mse_test]
}

gs_gbr2_perf_df = pd.DataFrame(gs_gbr2_perf_dict, index=['Train','Test'])
gs_gbr2_perf_df.T

gs_gbr2_R2_train: (1.0, 4)
gs_gbr2_R2_test: (1.0, 4)

gs_gbr2_mse_train: (4778.0, 4)
gs_gbr2_mse_test: (7860.0, 4)


,Train,Test
R2,0.718400,0.551165
MSE,4778.349247,7859.727603


In [68]:
gs_gbr2.best_estimator_

GradientBoostingRegressor(max_depth=6, max_features=200, n_estimators=400,
                          random_state=42)

In [ ]:
pd.DataFrame(gs_gbr2.cv_results_)

#### RandomizedSearchCV
[**Documentation on RandomizedSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [52]:
# build a randomized search for hyperparameters - killed after running 5 hours

gbr = GradientBoostingRegressor(random_state=42)

gbr_params = {
    'n_estimators': range(300, 501, 100),
    #'learning_rate':[0.01, 0.1],
    'max_depth': range(2, 5, 1),
    #'min_samples_split': [5,7,10],
    #'min_samples_leaf': [2,3,5],
    'max_features': range(200, 501, 100)
}

rs_gbr = RandomizedSearchCV(gbr,
                            gbr_params,
                            n_iter=50,
                            random_state=42,
                            cv=5)

# train the model

print(datetime.datetime.now())

rs_gbr.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-05 11:27:50.795222


KeyboardInterrupt: 

In [ ]:
# predict
rs_gbr_pred_train = rs_gbr.predict(X_train_ct)
rs_gbr_pred_test = rs.gbr.predict(X_test_ct)


# evaluate
rs_gbr_R2_train = rs_gbr.score(X_train_ct, y_train)
rs_gbr_R2_test = rs.gbr.score(X_test_ct, y_test)

print(f'rs_gbr_R2_train: {round(rs_gbr_R2_train), 4}')
print(f'rs_gbr_R2_test: {round(rs_gbr_R2_test), 4}')

rs_gbr_mse_train = metrics.mean_squared_error(y_train, rs_gbr_pred_train)
rs_gbr_mse_test = metrics.mean_squared_error(y_test, rs_gbr_pred_test)

rs_gbr_perf_dict = {
    'R2':  [rs_gbr_R2_train,  rs_gbr_R2_test],
    'MSE': [rs_gbr_mse_train, rs_gbr_mse_test]
}

rs_gbr_perf_df = pd.DataFrame(rs_gbr_perf_dict, index=['Train','Test'])
rs_gbr_perf_df.T

In [ ]:
rs_gbr.best_estimator_

In [ ]:
pd.DataFrame(rs_gbr.cv_results_)

#### BayesSearchCV
[**Documentation on skopt.BayesSearchCV**](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html)

In [ ]:
# build the parameter search CV

gbr = GradientBoostingRegressor(random_state=42)

gbr_params = {
    'n_estimators': [400, 600, 800],
    'learning_rate': [0.01, 0.1],
    'max_depth': [2,3,5,7],
    'min_samples_split': [5,7,10],
    'min_samples_leaf': [2,3,5],
    'max_features': [400, 600, 800, 1000] 
}

bs_gbr = BayesSearchCV(gbr,
                       gbr_params,
                       n_iter=32,
                       random_state=42)

# train
print(datetime.datetime.now())

bs_gbr.fit(X_train_ct, y_train)

print(datetime.datetime.now())

In [ ]:
# predict
bs_gbr_pred_train = bs_gbr.predict(X_train_ct)
bs_gbr_pred_test = bs_gbr.predict(X_teset_ct)

# evaluate
bs_gbr_R2_train = bs_gbr.score(X_train_ct, y_train)
bs_gbr_R2_test = bs_gbr.score(X_test_ct, y_test)

print(f'bs_gbr_R2_train: {round(bs_gbr_R2_train), 4}')
print(f'bs_gbr_R2_test: {round(bs_gbr_R2_test), 4}')
print()

bs_gbr_mse_train = metrics.mean_squared_error(X_train_ct, y_train)
bs_gbr_mse_test = metrics.mean_squared_error(X_test_ct, y_test)

print(f'bs_gbr_mse_train, {round(bs_gbr_mse_train), 4}')
print(f'bs_gbr_mse_test, {round(bs_gbr_mse_test), 4}')

In [ ]:
bs_gbr.best_estimator_

In [ ]:
pd.DataFrame(bs_gbr.cv_results_)

## Sileshi

## Mason

In [1]:
# imports
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer

In [2]:
# Loading dataset
df = pd.read_csv('../Data/salary_cleaned.csv')
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,year,month,year_month,timestamp_3mos,year_month_3mos,state_short,inflation_rate,inflation_rate_3mos,state,employment_rate,employment_rate_3mos
0,2018-06-03 13:58:20,Yahoo,IC2,Software Engineer,160.5,"Sunnyvale, CA",0.58,0.58,Full Stack,2018,6,2018-06,2018-03-03 13:58:20,2018-03,CA,0.029,0.024,California,0.957660,0.956797
1,2018-06-04 20:28:22,Facebook,E3,Software Engineer,165.0,"Seattle, WA",1.00,1.00,Full Stack,2018,6,2018-06,2018-03-04 20:28:22,2018-03,WA,0.029,0.024,Washington,0.955998,0.954978
2,2018-06-05 00:56:33,VmWare,Senior MTS,Software Engineer,218.0,"Palo Alto, CA",8.00,1.00,Distributed Systems (Back-End),2018,6,2018-06,2018-03-05 00:56:33,2018-03,CA,0.029,0.024,California,0.957660,0.956797
3,2018-06-05 01:19:05,Uber,L4,Software Engineer,240.0,"San Francisco, CA",3.00,0.00,Web Development (Front-End),2018,6,2018-06,2018-03-05 01:19:05,2018-03,CA,0.029,0.024,California,0.957660,0.956797
4,2018-06-05 07:13:17,Capital One,Master Software Engineer,Software Engineer,196.0,"New York, NY",8.00,2.00,iOS,2018,6,2018-06,2018-03-05 07:13:17,2018-03,NY,0.029,0.024,New York,0.959053,0.955962


In [3]:
# Converting Year and Month to string 
df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24496 entries, 0 to 24495
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                24496 non-null  object 
 1   company                  24496 non-null  object 
 2   level                    24496 non-null  object 
 3   title                    24496 non-null  object 
 4   totalyearlycompensation  24496 non-null  float64
 5   location                 24496 non-null  object 
 6   yearsofexperience        24496 non-null  float64
 7   yearsatcompany           24496 non-null  float64
 8   tag                      24496 non-null  object 
 9   year                     24496 non-null  object 
 10  month                    24496 non-null  object 
 11  year_month               24496 non-null  object 
 12  timestamp_3mos           24496 non-null  object 
 13  year_month_3mos          24496 non-null  object 
 14  state_short           

In [4]:
# Features and target variable
features = ['company', 'title', 'yearsofexperience', 'yearsatcompany', 'year', 'month', 'state_short', 'inflation_rate', 'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos']

X = df[features]
y = df['totalyearlycompensation']

In [5]:
# Train/Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

In [6]:
# ColumnTransformer
ct = ColumnTransformer([
    ('sc', StandardScaler(), make_column_selector(dtype_exclude=object)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'), ['company', 'title', 'state_short', 'year', 'month'])],
    n_jobs=-1,
    remainder='passthrough')

In [7]:
X_train_ct = ct.fit_transform(X_train)
X_test_ct = ct.transform(X_test)

In [8]:
X_train_ct.shape

(18372, 1234)

In [9]:
X_test_ct.shape

(6124, 1234)

#### RandomForest Regression

In [25]:
# Instantiate RandomForestRegression
rf = RandomForestRegressor(random_state=42)

# Parameters
rf_params={
    'n_estimators':[100, 200, 300, 400, 500],
    'criterion': ['mse'],
    'max_depth': [2, 4, 6, 8, 10, 12],
    'min_samples_split': [2, 3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4, 5],
}

# Instantiating RFR Gridsearch
rf_gs = GridSearchCV(rf, rf_params, cv=5, verbose=1, n_jobs=-1)

# Fitting GridSearch to the data
rf_gs.fit(X_train_ct, y_train)

Fitting 5 folds for each of 750 candidates, totalling 3750 fits


KeyboardInterrupt: 

In [24]:
# Train/Test Scores
print(f'Best Score: {rf_gs.best_score_}')
print(f'Train Score: {rf_gs.score(X_train_ct, y_train)}')
print(f'Test Score: {rf_gs.score(X_test_ct, y_test)}')

# Best Parameters
print(f'Best parameters: {rf_gs.best_params_}')

# Best Estimators
print(f'Best Estimators: {rf_gs.best_estimator_}')

Best Train Score: 0.41540868532825537
Test Score: 0.41070684312493977
Best parameters: {'criterion': 'mse', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 200}
Best Estimators: RandomForestRegressor(max_depth=6, min_samples_leaf=3, n_estimators=200,
                      random_state=42)


In [17]:
# Predictions
rf_preds_train = rf_gs.predict(X_train_ct)
rf_preds_test = rf_gs.predict(X_test_ct)

In [18]:
# Evaluation
rf_score_train = rf_gs.score(X_train_ct, y_train)
rf_score_test = rf_gs.score(X_test_ct, y_test)

rf_mse_train = metrics.mean_squared_error(y_train, rf_preds_train)
rf_mse_test = metrics.mean_squared_error(y_test, rf_preds_test)

In [19]:
# create the perf evaluation output df
perf_dict = {
    'R2': [rf_score_train, rf_score_test],
    'mse': [rf_mse_train, rf_mse_test]
}

perf_df = pd.DataFrame(perf_dict, index=['train','test']).T

# perf_df.to_csv('./randomforest_perf.csv')
# Data was obtained via AWS

In [20]:
# create the prediction output df
data_test_copy = X_test.copy()

In [21]:
# Predictions
data_test_copy['totalcomp'] = y_test
data_test_copy['predictions'] = rf_preds_test

# data_test_copy.to_csv('./randomforest_pred.csv', index=False)
# Data was obtained via AWS


In [22]:
# CV results
cv_result = pd.DataFrame(rf_gs.cv_results_)

# cv_result.to_csv('./randomforest_CV_Result.csv', index=False)
# Data was obtained via AWS

#### AdaBoost Regression

In [10]:
# Instantiate AdaBoostRegressor
abr = AdaBoostRegressor(random_state=42)

# Parameters
abr_params ={
    'base_estimator': [None],
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [1, 3],
    'loss': ['linear', 'square', 'exponential'],
}

# Instantiate ABR GridSearch
abr_gs = GridSearchCV(abr, abr_params, cv=5, verbose=1, n_jobs=-1)

# Fitting Gridsearch to the data
abr_gs.fit(X_train_ct, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=AdaBoostRegressor(random_state=42), n_jobs=-1,
             param_grid={'base_estimator': [None], 'learning_rate': [1, 3],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [50, 100, 150, 200]},
             verbose=1)

In [11]:
# Predictions
abr_preds_train = abr_gs.predict(X_train_ct)
abr_preds_test = abr_gs.predict(X_test_ct)

# Evaluation
abr_score_train = abr_gs.score(X_train_ct, y_train)
abr_score_test = abr_gs.score(X_test_ct, y_test)

abr_mse_train = metrics.mean_squared_error(y_train, abr_preds_train)
abr_mse_test = metrics.mean_squared_error(y_test, abr_preds_test)

In [13]:
# create the perf evaluation output df
abr_perf_dict = {
    'R2': [abr_score_train, abr_score_test],
    'mse': [abr_mse_train, abr_mse_test]
}

abr_perf_df = pd.DataFrame(abr_perf_dict, index=['train','test']).T

abr_perf_df.to_csv('../Data/adaboost_perf.csv')


In [14]:
# create the prediction output df
abrdata_test_copy = X_test.copy()

# Predictions
abrdata_test_copy['totalcomp'] = y_test
abrdata_test_copy['predictions'] = abr_preds_test

abrdata_test_copy.to_csv('../Data/adaboost_pred.csv', index=False)

In [15]:
# CV results
cv_result = pd.DataFrame(abr_gs.cv_results_)

cv_result.to_csv('../Data/adaboost_CV_Result.csv', index=False)